In [1]:
from database import * 
import numpy as np
import pandas as pd
import os
import ast
import matplotlib.pyplot as plt

type_all = execute_sql('select name from type', 1)[0]
actor_all = execute_sql('select name from actor', 1)[0]
director_all = execute_sql('select name from director', 1)[0]
month = [i for i in range(1, 13)]

W = [1, 6, 3.5, 0.5]


In [2]:

def binary_rep(list):
    return [
        [1 if item in list[0] else 0 for item in type_all],
        [1 if item in list[1] else 0 for item in actor_all],
        [1 if item in list[2] else 0 for item in director_all],
        list[3]
    ]
    
def month_diff(m1, m2):
    if m1 < m2:
        m1, m2 = m2, m1
    return 1 - min(m1 - m2, m2 + 12 - m1) / 12 

In [3]:
def similarity(A, B):
    res = []
    for i in range(3):
        res.append(np.dot(A[i], B[i])/(np.linalg.norm(A[i]) * np.linalg.norm(B[i])))
    res.append(month_diff(A[3], B[3]))
    
    # 加权平均
    return np.dot(W, res)


In [4]:
# TEST
similarity_1 = similarity(
    binary_rep(
            [
                ['动画', '剧情'], 
                ['吴京', '易烊千玺', '沈腾', '尼古拉斯·卡斯柯恩'],
                ['徐克', '姜文'],
                10
            ]),   
    binary_rep(
            [
                ['爱情', '剧情', '奇幻'], 
                ['吴京', '沈腾', '彭于晏', '伯纳德·希尔'],
                ['姜文', '贾玲'],
                8
            ])
    )
similarity_1


5.574914957130529

In [5]:
# 修改前:
# if not os.path.isfile('../csv/binary_data.csv'):
#     df = pd.read_csv('../csv/猫眼_豆瓣.csv')

# 修改后:
if not os.path.isfile('./csv/binary_data.csv'): # 修正路径
    df = pd.read_csv('./csv/猫眼_豆瓣_整合数据.csv') # 修正路径
    data = []
    for row in df.values:
        movie = []
        # 注意: row[1] 到 row[10] 是根据你之前提供的列名和索引推断的。
        # 如果你的实际猫眼_豆瓣.csv列顺序有变动，请调整这里的索引。
        # '电影名', '上映日期', '票房(万元)', '平均票价', '场均人数', '豆瓣评分', '豆瓣评论数', '导演', '演员', '类型', '地区', '语言', '时长'
        # name, date, money, director, actor, type = (row[1], row[2], row[3], row[8], row[9], row[10])
        # 根据你之前提供的列名：
        # 电影名 (0), 上映日期 (1), 票房(万元) (2), 平均票价 (3), 场均人数 (4), 豆瓣评分 (5), 豆瓣评论数 (6), 导演 (7), 演员 (8), 类型 (9), 地区 (10), 语言 (11), 时长 (12)
        # 应该是: name, date, money, director, actor, type = (row[0], row[1], row[2], row[7], row[8], row[9])
        # 请根据你的实际数据，如果列名是中文，可能需要用 df['列名'] 而不是 row[索引]
        name, date, money, director, actor, movie_type = (row[0], row[1], row[2], row[7], row[8], row[9])
        
        # 进一步检查 director, actor, movie_type 是否已经是列表类型，如果不是，需要 ast.literal_eval
        try:
            director_list = ast.literal_eval(director) if isinstance(director, str) else []
            actor_list = ast.literal_eval(actor) if isinstance(actor, str) else []
            movie_type_list = ast.literal_eval(movie_type) if isinstance(movie_type, str) else []
        except (ValueError, SyntaxError):
            print(f"Warning: Could not parse list for movie {name}. Using empty list.")
            director_list = []
            actor_list = []
            movie_type_list = []

        year, month, day = date.split('-')
        
        # 修正 binary_rep 函数的调用，传递解析后的列表
        movie = binary_rep([movie_type_list, actor_list, director_list, int(month)])
        
        # print(movie)
        movie.append(money)
        movie.append(int(year))
        movie.append(name)
        data.append(movie)

    data = pd.DataFrame(data, columns=['type', 'actor', 'director', 'month', 'money', 'year', 'name'])
    data.to_csv('./csv/binary_data.csv', index=False) # 修正路径，并添加 index=False

In [6]:
from sklearn.linear_model import LinearRegression 


sql =  'select  substr(date,1,4)  year, avg(money) count \
        from type t left outer join maoyan_movie_type mt on t.id = mt.type_id join maoyan_movie m on m.id = mt.maoyan_id\
        group by  year\
        order by  year\
        limit 1, 14'
x, y = execute_sql(sql, 2)
y = [float(i) for i in y]
x = np.array(x,dtype=int)
y = np.array(y,dtype=float)

reg = LinearRegression().fit(x.reshape(-1, 1), y.reshape(-1, 1))


plt.figure(figsize=(12, 3.5))
plt.scatter(x, y,  color='black')
plt.plot(x, reg.predict(x.reshape(-1, 1)), color='red', linewidth=1)
plt.show()




OperationalError: (1054, "Unknown column 'date' in 'field list'")

In [ ]:
# 修改前:
# data = pd.read_csv('../csv/binary_data.csv')

# 修改后:
def predict(movie):
    movie = binary_rep(movie)
    data = pd.read_csv('./csv/binary_data.csv') # 修正路径
    
    res = []
    for item in data.values:
        # 这里也需要 ast.literal_eval 来正确解析字符串列表
        type_data = ast.literal_eval(item[1]) if isinstance(item[1], str) else []
        actor_data = ast.literal_eval(item[2]) if isinstance(item[2], str) else []
        director_data = ast.literal_eval(item[3]) if isinstance(item[3], str) else []
        
        month, money, year, name = item[4], int(item[5]), int(item[6]), item[7]
        
        # 传递解析后的列表
        res.append((similarity(movie, [type_data, actor_data, director_data, month]), money, year, name))
    
    res.sort(key=lambda x : -x[0])
    
    predict_val = 0
    cnt = 0
    for i in range(5):
        if res[i][0] <= 2:
            break
        cnt += 1
        predict_val += res[i][1] * reg.predict([[2023]]) / reg.predict([[res[i][2]]])
        
        print(res[i])
        
    return predict_val[0][0] / cnt


In [ ]:
movie = [
    ['剧情', '历史'],
    ['吴京', '包贝尔', '易烊千玺', '邓超', '欧豪', '雷佳音', '郭京飞'],
    ['徐克', '吴京'],
    2
]

predict(movie)

In [ ]:
movie = [
    ['爱情', '喜剧'],
    ['韩庚', '郑恺', '于文文', '刘雅瑟', '张天爱'],
    ['田羽生'],
    10
]

predict(movie)